In [ ]:
# # 1. 필수 라이브러리 설치
# !pip install -q -U google-generativeai


In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [ ]:

import google.generativeai as genai
import pandas as pd
import json
import time
import os
from tqdm import tqdm
from google.colab import drive

# ======================================================
# [설정 1] API 키 입력 (따옴표 안에 붙여넣기)
API_KEY = os.getenv('API_KEY')
genai.configure(api_key=API_KEY)

# [설정 2] 모델 설정 (Gemini 1.5 Flash)
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config={"response_mime_type": "application/json"}
)

# [설정 3] 구글 드라이브 마운트 (데이터 보호를 위해 필수!)
drive.mount('/content/drive')

# ======================================================
# 2. 데이터 불러오기 (CSV)
# ※ 파일 경로를 본인 껄로 수정하세요!
csv_path = '/content/drive/MyDrive/news_data.csv' 

print("데이터를 불러오는 중...")
try:
    df = pd.read_csv(csv_path) # 한글 깨지면: encoding='cp949' 또는 'euc-kr' 추가
except:
    df = pd.read_csv(csv_path, encoding='cp949')

print(f"총 데이터 개수: {len(df)}건")

# ★ 중요: 데이터 칼럼 이름 맞추기 (본인 CSV 헤더 확인 필요)
# 예: 내 CSV의 제목 칼럼이 'article_title'이면 -> 'title'로 변경
# df = df.rename(columns={'article_title': 'title', 'article_content': 'body'})
# 만약 이미 'title', 'body'라는 이름이 없으면 아래 줄을 수정해서 매핑해주세요.
# 여기서는 편의상 첫 번째 칼럼을 title, 두 번째를 body로 가정하거나
# 'title', 'body' 칼럼이 있다고 가정합니다.
# df['id'] = df.index  # ID가 없으면 인덱스를 ID로 사용

# 데이터가 너무 많으므로 테스트를 위해 처음 600개만 먼저 돌려보시는 걸 추천
# df = df[:600]  # 주석 제거하면 600개만 테스트

# ======================================================
# 3. 프롬프트 생성 함수 (Batch Size 300 최적화)
def make_prompt(chunk_data):
    # API로 보낼 때는 가볍게 ID, 제목, 본문만 추려서 보냄
    simplified_data = []
    for item in chunk_data:
        simplified_data.append({
            "id": item['id'], 
            "title": item['title'], 
            "body": str(item['body'])[:200] # 본문은 앞 200자만 (토큰 절약)
        })
    
    news_list_str = json.dumps(simplified_data, ensure_ascii=False)
    
    return f"""
    너는 금융 뉴스 감성 분석기야. 
    아래 뉴스 목록(JSON)을 보고 KOSPI200 기업에 대한 '호재', '악재', '중립'을 판단해.
    
    [제약사항]
    1. 결과는 오직 JSON 리스트로만 출력.
    2. 이유 설명 금지. (속도를 위해)
    3. Sentiment는 "호재", "악재", "중립" 중 하나.
    
    [입력 데이터]
    {news_list_str}
    
    [출력 예시]
    [{{"id": 0, "sentiment": "호재"}}, {{"id": 1, "sentiment": "중립"}}]
    """

# ======================================================
# 4. 실행 로직 (Batch Processing)
BATCH_SIZE = 300  # 한 번에 300개씩 (하루 126만 건 처리 속도)
results = []
save_interval = 10 # 10번 요청(3000건)마다 중간 저장

# DataFrame을 딕셔너리 리스트로 변환 (ID 확보)
if 'id' not in df.columns:
    df['id'] = df.index
data_records = df.to_dict('records')

chunks = [data_records[i:i + BATCH_SIZE] for i in range(0, len(data_records), BATCH_SIZE)]

print(f"시작합니다! 총 {len(chunks)}번의 요청을 보냅니다.")

for i, chunk in enumerate(tqdm(chunks)):
    try:
        prompt = make_prompt(chunk)
        
        # API 호출
        response = model.generate_content(prompt)
        
        # 결과 파싱
        json_result = json.loads(response.text)
        results.extend(json_result)
        
        # 중간 저장 (혹시 끊길까봐)
        if (i + 1) % save_interval == 0:
            temp_df = pd.DataFrame(results)
            temp_df.to_csv(f'/content/drive/MyDrive/news_result_temp.csv', index=False)
        
        # 무료 등급 제한 고려 (안전하게 10초 대기)
        # 300개 처리에 10초 쉬면 분당 1800개 처리 속도 -> 아주 훌륭함
        time.sleep(10) 

    except Exception as e:
        print(f"⚠️ {i}번째 묶음 에러 발생: {e}")
        # 에러 나도 멈추지 않고 다음으로 넘어감 (나중에 빈 곳만 다시 채우면 됨)
        time.sleep(20) # 에러 났을 땐 좀 더 오래 대기

# ======================================================
# 5. 최종 결과 저장 및 병합
print("분석 완료! 결과를 원본 데이터와 합칩니다.")

result_df = pd.DataFrame(results)
# 원본 데이터와 'id' 기준으로 합치기
final_df = pd.merge(df, result_df, on='id', how='left')

# 저장
final_path = '/content/drive/MyDrive/final_news_analysis.csv'
final_df.to_csv(final_path, index=False)

print(f"저장 완료: {final_path}")
print(final_df[['title', 'sentiment']].head())

In [7]:
import pandas as pd

In [8]:
import os

In [9]:
file_list = os.listdir('../newsdata/')

In [6]:
file_list

['BGF리테일.csv',
 'BNK금융지주.csv',
 'CJ.csv',
 'CJ대한통운.csv',
 'CJ제일제당.csv',
 'DB손해보험.csv',
 'DGB금융지주.csv',
 'DL.csv',
 'DL이앤씨.csv',
 'F&F.csv',
 'GKL.csv',
 'GS.csv',
 'GS건설.csv',
 'GS리테일.csv',
 'HD한국조선해양.csv',
 'HD현대.csv',
 'HD현대미포.csv',
 'HD현대인프라코어.csv',
 'HD현대일렉트릭.csv',
 'HD현대중공업.csv',
 'HL만도.csv',
 'HMM.csv',
 'HS효성첨단소재.csv',
 'JB금융지주.csv',
 'KB금융.csv',
 'KCC.csv',
 'KG모빌리티.csv',
 'KT&G.csv',
 'KT.csv',
 'LG.csv',
 'LG디스플레이.csv',
 'LG생활건강.csv',
 'LG에너지솔루션.csv',
 'LG유플러스.csv',
 'LG이노텍.csv',
 'LG전자.csv',
 'LG화학.csv',
 'LIG넥스원.csv',
 'LS ELECTRIC.csv',
 'LS.csv',
 'LX인터내셔널.csv',
 'NAVER.csv',
 'NH투자증권.csv',
 'OCI.csv',
 'OCI홀딩스.csv',
 'PI첨단소재.csv',
 'POSCO홀딩스.csv',
 'S-Oil.csv',
 'SK.csv',
 'SKC.csv',
 'SK네트웍스.csv',
 'SK바이오사이언스.csv',
 'SK바이오팜.csv',
 'SK스퀘어.csv',
 'SK아이이테크놀로지.csv',
 'SK이노베이션.csv',
 'SK케미칼.csv',
 'SK텔레콤.csv',
 'SK하이닉스.csv',
 'TCC스틸.csv',
 'TKG휴켐스.csv',
 '강원랜드.csv',
 '고려아연.csv',
 '금양.csv',
 '금호석유.csv',
 '금호타이어.csv',
 '기아.csv',
 '기업은행.csv',
 '넷마블.csv',
 '녹십자.csv',
 '녹십자홀딩스.cs

In [14]:
df = pd.read_csv('../newsdata/CJ.csv')

In [15]:
df

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",1040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,1040
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,1040
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,1040
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,1040
...,...,...,...,...
2609,20241219,"[리포트 브리핑]CJ, '실적 개선으로 주주환원이 확대될 자회사 가치' 목표가 16...",[서울=뉴스핌] 로보뉴스 = BNK투자증권에서 19일 CJ(001040)에 대해 '...,1040
2610,20241219,"마이크로바이옴 선두주자 '3사 3색', 누가 유리할까",이 기사는 2024년12월12일 09시05분에 팜이데일리 프리미엄 콘텐츠로 선공개 ...,1040
2611,20241223,마트 외식업 등 포인트 유효기간 연장 신세계 CJ '3년'까지,[세종=이데일리 하상렬 기자] 앞으로 대형마트 편의점 외식 등 8개 업종의 적립식 ...,1040
2612,20241225,"[현장에서]축제 분위기 넷플릭스, 진퇴양난 토종OTT",[이데일리 윤정훈 기자] 계엄령으로 차분해진 연말 도심에 핫핑크 물결이 일고 있다....,1040


In [ ]:
import google.generativeai as genai
import pandas as pd
import time
import json
import os
from tqdm import tqdm
from google.colab import drive

# ======================================================
# [설정 1] API 키 입력 (본인 키로 변경!)
API_KEY = os.getenv('API_KEY')
genai.configure(api_key=API_KEY)

# [설정 2] 모델 설정 (Gemini 1.5 Flash)
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config={"response_mime_type": "application/json"}
)

# [설정 3] 구글 드라이브 마운트
drive.mount('/content/drive')

# ======================================================
# 2. 데이터 불러오기
# ※ 파일 경로 수정 필요
csv_path = '/content/drive/MyDrive/news_data.csv' 

print("데이터를 불러옵니다...")
try:
    df = pd.read_csv(csv_path)
except:
    df = pd.read_csv(csv_path, encoding='cp949')

# ★ 컬럼 이름 맞추기 (본인 파일 헤더 확인 필수)
# df = df.rename(columns={'기사제목': 'title', '본문': 'body'})

# [하루 제한] 1,500개만 잘라서 작업 (원본 데이터 프레임 카피)
# 슬라이싱 [:1500] -> 0번부터 1499번까지
target_df = df[0].copy()

print(f"총 {len(df)}건 중 앞부분 {len(target_df)}건을 분석합니다.")
print("예상 소요 시간: 약 1시간 40분 (무료 등급 속도 제한 준수)")

# ======================================================
# 3. 분석 루프 시작 (For문)
results_list = [] # 결과를 담을 빈 리스트
save_interval = 50 # 50개마다 중간 저장

print("🚀 분석 시작!")

# tqdm으로 진행률 바 표시
for index, row in tqdm(target_df.iterrows(), total=len(target_df)):
    try:
        # 1) 데이터 준비 (본문 200자 제한 반영)
        title = str(row['title'])
        body = str(row['body'])[:200] # 요청하신 대로 200자 컷

        # 2) 프롬프트 작성
        prompt = f"""
        너는 금융 뉴스 분석가야. 아래 뉴스를 보고 KOSPI200 기업 입장에서 호재/악재/중립을 판단해.

        [뉴스 정보]
        - 제목: {title}
        - 본문: {body}

        [분석 요청]
        1. sentiment: "호재", "악재", "중립" 중 택1
        2. reason: 판단 이유를 한 문장으로 요약

        응답은 반드시 JSON 형식으로: {{"sentiment": "결과", "reason": "이유"}}
        """

        # 3) API 호출
        response = model.generate_content(prompt)
        
        # 4) 결과 해석
        json_res = json.loads(response.text)
        
        # 5) 결과 저장 (원본의 index를 키값으로 저장)
        results_list.append({
            "original_index": index,     # 나중에 원본과 합칠 때 사용하는 열쇠
            "sentiment": json_res.get("sentiment"),
            "reason": json_res.get("reason")
        })

        # ★ 4초 대기 (무료 제한 준수)
        time.sleep(4)
        
        # 6) 중간 저장 (혹시 꺼질까봐)
        if len(results_list) % save_interval == 0:
            pd.DataFrame(results_list).to_csv('temp_results.csv', index=False)

    except Exception as e:
        print(f"⚠️ {index}번 에러 발생: {e}")
        # 에러 나면 비워두고 진행 (나중에 확인 가능)
        results_list.append({
            "original_index": index,
            "sentiment": "Error",
            "reason": str(e)
        })
        time.sleep(5)

# ======================================================
# 4. 결과 합치기 (Merge)
print("분석 완료! 원본 데이터에 결과를 붙입니다...")

# 결과 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(results_list)

# 원본(target_df)에 결과(result_df)를 인덱스 기준으로 합치기
# left_index=True: 원본의 인덱스 사용
# right_on='original_index': 결과의 original_index 사용
final_df = pd.merge(target_df, result_df, left_index=True, right_on='original_index', how='left')

# 불필요한 중복 인덱스 컬럼 정리
if 'original_index' in final_df.columns:
    final_df = final_df.drop(columns=['original_index'])

# ======================================================
# 5. 최종 파일 저장
save_path = '/content/drive/MyDrive/news_analysis_completed.csv'
final_df.to_csv(save_path, index=False)

print(f"🎉 저장 완료! 경로: {save_path}")
print("\n[결과 미리보기]")
print(final_df[['title', 'sentiment', 'reason']].head())

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# 파일 불러오기
file_name = 'news_data.csv'  # 파일명 확인!

if not os.path.exists(file_name):
    print("❌ 에러: 파일을 찾을 수 없습니다. 같은 폴더에 있는지 확인하세요.")
else:
    try:
        df = pd.read_csv(file_name)
    except:
        df = pd.read_csv(file_name, encoding='cp949')
    
    # ★ 컬럼 이름 맞추기 (필요시 수정)
    # df = df.rename(columns={'기사제목': 'title', '본문내용': 'body'})

    # 1500개 자르기
    target_df = df[:1500].copy()
    
    print(f"✅ 데이터 로드 성공! 총 {len(df)}건 중 {len(target_df)}건 분석 예정")
    # 데이터 눈으로 확인 (상위 3개)
    display(target_df.head(3))

In [13]:
target_df = df.loc[0]

In [14]:
target_df

일자                                                 20150101
제목                              유통 식음료 업계 신년 화두는 `혁신과 수익창출`
본문        [이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...
ticker                                                 1040
Name: 0, dtype: object

In [ ]:
target_df.rename(col)

In [ ]:


import google.generativeai as genai
import pandas as pd
import time
import json
import os
from tqdm.notebook import tqdm

# ==========================================
# ★ API 키 입력 (여기에 본인 키를 넣으세요)
API_KEY = os.getenv('API_KEY')
genai.configure(api_key=API_KEY)

# 모델 설정 (가성비 좋은 1.5 Flash)
model = genai.GenerativeModel(
    'gemini-2.5-flash',
    generation_config={"response_mime_type": "application/json"}
)

print("✅ 설정 완료!")

✅ 설정 완료!


In [27]:
# 파일 이름 (같은 폴더에 있어야 함)
file_name = 'news_data.csv' 

# 1. 파일 읽기
if not os.path.exists(file_name):
    print(f"❌ 에러: '{file_name}' 파일이 없습니다.")
else:
    try:
        df = pd.read_csv(file_name)
    except:
        df = pd.read_csv(file_name, encoding='cp949') # 한글 깨짐 방지

    # 2. 데이터 확인 (일자, 제목, 본문, ticker 컬럼이 잘 있는지)
    print(f"📋 컬럼 목록: {list(df.columns)}")
    
    # [중요] 무료 등급 하루 제한(1500개)에 맞춰 자르기
    target_df = df[:1500].copy()
    
    print(f"✅ 총 {len(df)}건 중 앞부분 {len(target_df)}건을 분석합니다.")
    display(target_df.head(2)) # 데이터 눈으로 확인

❌ 에러: 'news_data.csv' 파일이 없습니다.


In [ ]:
target_df = df.loc[:150]

In [29]:
target_df

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",1040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,1040


In [30]:

import google.generativeai as genai
import pandas as pd
import time
import json
import os
from tqdm.notebook import tqdm

In [37]:
results_list = []
save_interval = 50 

print("🚀 분석 시작! (약 1시간 40분 소요 예정)")

# tqdm으로 진행률 표시
for index, row in tqdm(target_df.iterrows(), total=len(target_df)):
    try:
        # ★ 여기서 '제목'과 '본문' 컬럼만 쏙 뽑습니다.
        # (ticker나 일자는 분석에 안 쓰고 결과에만 남깁니다)
        title = str(row['제목']) 
        body = str(row['본문'])[:200] # 본문은 200자만 (비용 절약)
        ticker = str(row['ticker'])   # 참고용 (프롬프트에 넣을 수도 있음)

        # 프롬프트 작성
        prompt = f"""
        너는 주식 데이터 분석가야. 
        아래 뉴스를 보고 해당 종목({ticker})에 대해 '호재', '악재', '중립'을 판단해.

        [뉴스 정보]
        - 제목: {title}
        - 본문: {body}

        [분석 요청]
        1. sentiment: "호재", "악재", "중립" 중 택1를 하되 '중립'이면 0, '호재'이면 1, '악재'이면 -1로 해
        2. reason: 판단 이유를 한 문장으로 요약

        응답은 반드시 JSON 형식으로: {{"sentiment": "결과", "reason": "이유"}}
        """

        # API 호출
        response = model.generate_content(prompt)
        json_res = json.loads(response.text)
        
        # 결과 리스트에 담기 (나중에 합치기 위해 original_index 저장)
        results_list.append({
            "original_index": index,
            "sentiment": json_res.get("sentiment"),
            "reason": json_res.get("reason")
        })

        # ★ 4초 대기 (무료 제한: 1분 15회 준수)
        time.sleep(1)
        
        # 중간 저장 (혹시 꺼질까봐)
        if len(results_list) % save_interval == 0:
            pd.DataFrame(results_list).to_csv('temp_results.csv', index=False)

    except Exception as e:
        print(f"⚠️ {index}번 에러: {e}")
        results_list.append({"original_index": index, "sentiment": "Error", "reason": str(e)})
        time.sleep(5)

print("🎉 분석 끝! 다음 셀을 실행하세요.")

🚀 분석 시작! (약 1시간 40분 소요 예정)


  0%|          | 0/2 [00:00<?, ?it/s]

🎉 분석 끝! 다음 셀을 실행하세요.


In [38]:
# 1. 결과 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(results_list)

# 2. 원본(일자, 제목, 본문, ticker) + 결과(sentiment, reason) 합치기
final_df = pd.merge(target_df, result_df, left_index=True, right_on='original_index', how='left')

# 3. 불필요한 인덱스 컬럼 제거
if 'original_index' in final_df.columns:
    final_df = final_df.drop(columns=['original_index'])

# 4. 저장 (한글 안 깨지게 utf-8-sig)
save_path = 'news_analysis_completed.csv'
final_df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"💾 저장 완료: {save_path}")
print("\n[최종 결과 미리보기]")
# 일자, ticker, 제목, 결과만 딱 보여주기
display(final_df[['일자', 'ticker', '제목', 'sentiment']].head())

💾 저장 완료: news_analysis_completed.csv

[최종 결과 미리보기]


,일자,ticker,제목,sentiment
0,20150101,1040,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,0
1,20150102,1040,[투자의맥]박스권 상단 넘어선 종목에 주목,0


In [33]:
df

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",1040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,1040
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,1040
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,1040
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,1040
...,...,...,...,...
2609,20241219,"[리포트 브리핑]CJ, '실적 개선으로 주주환원이 확대될 자회사 가치' 목표가 16...",[서울=뉴스핌] 로보뉴스 = BNK투자증권에서 19일 CJ(001040)에 대해 '...,1040
2610,20241219,"마이크로바이옴 선두주자 '3사 3색', 누가 유리할까",이 기사는 2024년12월12일 09시05분에 팜이데일리 프리미엄 콘텐츠로 선공개 ...,1040
2611,20241223,마트 외식업 등 포인트 유효기간 연장 신세계 CJ '3년'까지,[세종=이데일리 하상렬 기자] 앞으로 대형마트 편의점 외식 등 8개 업종의 적립식 ...,1040
2612,20241225,"[현장에서]축제 분위기 넷플릭스, 진퇴양난 토종OTT",[이데일리 윤정훈 기자] 계엄령으로 차분해진 연말 도심에 핫핑크 물결이 일고 있다....,1040


In [4]:
!pip install openai

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 1.7 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- ------------------- 1/2 [openai]
   -------------------- -

In [49]:
# 1. 라이브러리 강제 업데이트 (404 에러 해결의 핵심!)
# 실행 후 "Restart Kernel" 버튼이 뜨면 눌러주세요.

import google.generativeai as genai
import pandas as pd
import time
import json
import os
from tqdm.notebook import tqdm

# 2. API 키 설정 (본인 키 확인!)
API_KEY = os.getenv('API_KEY')
genai.configure(api_key=API_KEY)

# 3. Role(역할) 정의 - 사용자가 원했던 기능!
# 이 설정이 들어가면 모든 대답을 이 페르소나로 대답합니다.
ROLE_DEFINITION = """
너는 월스트리트에서 20년 경력을 가진 '퀀트 투자 전문가(Quant Analyst)'야.
너의 임무는 뉴스 기사를 읽고 주가에 미칠 영향을 냉철하게 분석하여 점수를 매기는 거야.
감정에 치우치지 말고, 오직 데이터와 팩트에 기반해서 판단해.
"""

# ==========================================
# 2. [핵심] 사용 가능한 모델 자동 찾기 (404 에러 박멸)
valid_model_name = None

print("📡 사용 가능한 모델을 찾는 중...")
try:
    for m in genai.list_models():
        # 'generateContent'를 지원하고 이름에 'flash'가 들어가는 최신 모델 찾기
        if 'generateContent' in m.supported_generation_methods:
            if 'flash' in m.name:
                valid_model_name = m.name
                print(f"✅ 찾았다! 이 모델을 사용합니다: {valid_model_name}")
                break
    
    # 만약 flash가 없으면 pro 모델 찾기
    if not valid_model_name:
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods and 'pro' in m.name:
                valid_model_name = m.name
                print(f"⚠️ Flash 모델을 못 찾아서 Pro 모델로 대체합니다: {valid_model_name}")
                break

    # 모델 설정
    model = genai.GenerativeModel(
        valid_model_name,
        generation_config={"response_mime_type": "application/json"}
    )

except Exception as e:
    print(f"❌ 모델 목록을 가져오는 데 실패했습니다. API 키를 확인해주세요.\n에러: {e}")
    exit()

📡 사용 가능한 모델을 찾는 중...
✅ 찾았다! 이 모델을 사용합니다: models/gemini-2.5-flash


In [41]:
df = df[:150]

In [42]:
df

,일자,제목,본문,ticker
0,20150101,유통 식음료 업계 신년 화두는 `혁신과 수익창출`,"[이데일리 민재용 기자] 유통 식음료 업체들은 올해 변화와 혁신, 사업 경쟁력 제고...",1040
1,20150102,[투자의맥]박스권 상단 넘어선 종목에 주목,[이데일리 경계영 기자] 대신증권은 2일 올해도 개별 종목 장세가 이어지는 가운데 ...,1040
2,20150108,"CJ프레시웨이, 종합인증우수업체 재공인 획득",[이데일리 염지현 기자] CJ(001040)그룹의 식자재 유통 및 단체급식 전문기업...,1040
3,20150108,올해 창조경제 화두는 '안전'..사회안전과 신시장 창출 노린다,[이데일리 이승현 기자] 정부가 올해 창조경제의 새 화두로 ‘안전’을 내세우며 사회...,1040
4,20150108,"이미경 CJ 부회장, 동생 복귀 도우려 경영서 손 떼나?",[이데일리 민재용 기자] 이미경 CJ(001040)그룹 부회장이 건강상의 이유로 미...,1040
...,...,...,...,...
145,20150709,"CJ그룹, 협력업체와 상생..납품대금 조기 지급",[이데일리 함정선 기자] CJ(001040)그룹은 협력업체와 상생을 위해 7월 납품...,1040
146,20150709,[마감]코스피 널뛰기후 상승 `대장주` 삼성전자 힘냈다,[이데일리 송이라 기자] ‘롤러코스터’라는 표현이 딱 들어맞는 하루였다. 코스피가 ...,1040
147,20150710,[마감]코스피 상승 中 증시 상승 효과,[이데일리 송이라 기자] 코스피가 이틀째 상승했다. 중국 증시가 5% 이상 오름세를...,1040
148,20150713,"국민연금, 경기방어주 사고 수출주 팔았다",국내 증시의 '큰손' 국민연금이 지난 2 4분기 중 필수 소비재와 같은 경기 방어주...,1040


In [48]:
# 파일 불러오기
file_name = '../newsdata/CJ.csv' 

if not os.path.exists(file_name):
    print("❌ 파일을 찾을 수 없습니다. 같은 폴더에 있는지 확인하세요.")
else:
    # 데이터 읽기
    try:
        df = pd.read_csv(file_name)
    except:
        df = pd.read_csv(file_name, encoding='cp949')

    # 1500개 자르기
    target_df = df[:50].copy()
    print(f"데이터 준비 완료: {len(target_df)}건")

    # 분석 시작
    results_list = []
    print("🚀 퀀트 분석 시작! (Role: 20년차 전문가)")

    for index, row in tqdm(target_df.iterrows(), total=len(target_df)):
        try:
            # 제목, 본문, 종목명 추출
            title = str(row.get('제목', '')) # 컬럼명 확인 필요
            body = str(row.get('본문', ''))[:300]
            ticker = str(row.get('ticker', '해당 종목'))

            # 프롬프트 (Role은 이미 위에서 설정했으므로, 여기선 할 일만 시킴)
            prompt = f"""
            [분석 대상 종목: {ticker}]
            
            [뉴스 데이터]
            제목: {title}
            본문: {body}

            [수행 작업]
            위 뉴스를 보고 호재(1), 중립(0), 악재(-1) 중 하나로 평가해.
            
            [출력 포맷(JSON)]
            {{"score": 정수값, "reason": "이유"}}
            """

            # 실행
            response = model.generate_content(prompt)
            json_res = json.loads(response.text)
            
            results_list.append({
                "original_index": index,
                "score": json_res.get("score"),
                "reason": json_res.get("reason")
            })

            time.sleep(7) # 무료 제한 준수

            # 중간 저장 (50개마다)
            if len(results_list) % 50 == 0:
                pd.DataFrame(results_list).to_csv('temp_results.csv', index=False)

        except Exception as e:
            print(f"⚠️ {index}번 에러: {e}")
            time.sleep(5)

    # 최종 저장
    final_df = pd.merge(target_df, pd.DataFrame(results_list), left_index=True, right_on='original_index', how='left')
    if 'original_index' in final_df.columns:
        final_df = final_df.drop(columns=['original_index'])
        
    final_df.to_csv('final_quant_analysis.csv', index=False, encoding='utf-8-sig')
    print("🎉 저장 완료: final_quant_analysis.csv")
    display(final_df.head())

데이터 준비 완료: 50건
🚀 퀀트 분석 시작! (Role: 20년차 전문가)


  0%|          | 0/50 [00:00<?, ?it/s]

⚠️ 0번 에러: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 45.248654328s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 45
}
]
⚠️ 1번 에러: 429 You exceeded your current quota, please check your plan and billing details

KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd
import json
import time
import google.generativeai as genai
from tqdm import tqdm

# ==========================================
# 1. API 키 설정 (본인 키 확인!)
API_KEY = os.getenv('API_KEY')
genai.configure(api_key=API_KEY)

# ==========================================
# 2. [핵심] 사용 가능한 모델 자동 찾기 (404 에러 박멸)
valid_model_name = None

print("📡 사용 가능한 모델을 찾는 중...")
try:
    for m in genai.list_models():
        # 'generateContent'를 지원하고 이름에 'flash'가 들어가는 최신 모델 찾기
        if 'generateContent' in m.supported_generation_methods:
            if 'flash' in m.name:
                valid_model_name = m.name
                print(f"✅ 찾았다! 이 모델을 사용합니다: {valid_model_name}")
                break
    
    # 만약 flash가 없으면 pro 모델 찾기
    if not valid_model_name:
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods and 'pro' in m.name:
                valid_model_name = m.name
                print(f"⚠️ Flash 모델을 못 찾아서 Pro 모델로 대체합니다: {valid_model_name}")
                break

    # 모델 설정
    model = genai.GenerativeModel(
        valid_model_name,
        generation_config={"response_mime_type": "application/json"}
    )

except Exception as e:
    print(f"❌ 모델 목록을 가져오는 데 실패했습니다. API 키를 확인해주세요.\n에러: {e}")
    exit()

# ==========================================
# 3. 데이터 로드 및 분석 설정
file_path = '../newsdata/CJ.csv' # 경로 확인!
file_name_only = os.path.basename(file_path).split('.')[0]

if not os.path.exists(file_path):
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
else:
    # 데이터 읽기
    try:
        df = pd.read_csv(file_path)
    except:
        df = pd.read_csv(file_path, encoding='cp949')

    target_df = df[:150].copy() # 150개 테스트
    print(f"📊 데이터 {len(target_df)}건 로드 완료. 분석을 시작합니다.")

    results_list = []
    
    for index, row in tqdm(target_df.iterrows(), total=len(target_df)):
        try:
            title = str(row.get('제목', ''))
            body = str(row.get('본문', ''))[:300]
            ticker = str(row.get('ticker', file_name_only))

            prompt = f"""
            [종목: {ticker}]
            뉴스 제목: {title}
            뉴스 본문: {body}
            
            위 뉴스를 보고 이 종목에 대해 호재(1), 중립(0), 악재(-1) 중 하나로 점수를 매겨줘.
            응답은 JSON 포맷: {{"score": 정수, "reason": "한줄요약"}}
            """

            # 실행
            response = model.generate_content(prompt)
            
            # 파싱 보정
            clean_text = response.text.replace('```json', '').replace('```', '').strip()
            json_res = json.loads(clean_text)
            
            results_list.append({
                "original_index": index,
                "score": json_res.get("score"),
                "reason": json_res.get("reason")
            })

            time.sleep(4) # 무료 제한 준수

            # 중간 저장
            if len(results_list) % 50 == 0:
                pd.DataFrame(results_list).to_csv(f'temp_{file_name_only}.csv', index=False)

        except Exception as e:
            print(f"⚠️ {index}번 에러: {e}")
            time.sleep(5)

    # 최종 저장
    final_df = pd.merge(target_df, pd.DataFrame(results_list), left_index=True, right_on='original_index', how='left')
    if 'original_index' in final_df.columns:
        final_df = final_df.drop(columns=['original_index'])
        
    save_name = f'{file_name_only}_quant_result.csv'
    final_df.to_csv(save_name, index=False, encoding='utf-8-sig')
    print(f"🎉 저장 완료: {save_name}")